
To check cam view while trainng RL model,
make sure that train.ipynb is running


In [1]:
import sys
sys.path.append('/home/kaustubh/Documents/GitHub/Self_Driving_Car/')

In [1]:
import carla
import numpy as np
# from Imitation_Learning_RL.process_frame import process_frame

In [2]:
seg_image = None
rgb_image = None
collision = None
lane_invasion = None

In [3]:
# Callbacks
def image_callback(data):
    global rgb_image, seg_image
    array = np.frombuffer(data.raw_data, dtype=np.uint8)
    array = np.reshape(array, (data.height, data.width, 4))[:, :, :3]
    rgb_image = array
    # seg_image = process_frame(rgb_image)

def collision_callback(data):
    global collision
    collision = True

def lane_invasion_callback(event):
    global lane_invasion
    lane_invasion = True

In [4]:
# client_ = carla.Client('localhost', 2000)
client_ = carla.Client("192.168.0.2", 2030)
client_.set_timeout(10.0)
world_ = client_.get_world()
blueprint_library_ = world_.get_blueprint_library()
vehicle_bp = blueprint_library_.find("vehicle.mini.cooper_s")
spawn_points = world_.get_map().get_spawn_points()
vehicle = world_.try_spawn_actor(vehicle_bp, spawn_points[0])

In [5]:
# vehicle = world_.get_actors().filter("vehicle.volkswagen.t2_2021")
# vehicle = world_.get_actors().filter("vehicle.mini.cooper_s")[0]
print(vehicle)

# camera_sensor = world_.get_actors().filter("sensor.camera.rgb")[0]
# collision_sensor = world_.get_actors().filter("sensor.other.collision")[0]
# lane_invasion_sensor = world_.get_actors().filter("sensor.other.lane_invasion")[0]

Actor(id=24, type=vehicle.mini.cooper_s)


In [6]:
camera_bp_ = blueprint_library_.find('sensor.camera.rgb')
collision_bp = blueprint_library_.find("sensor.other.collision")
lane_inv_bp = blueprint_library_.find("sensor.other.lane_invasion")

camera_bp_.set_attribute('image_size_x', '128')
camera_bp_.set_attribute('image_size_y', '128')


camera_location = carla.Location(x=4.442184 / 2.9, y=0, z=2.2)

camera_transform = carla.Transform(camera_location)
collision_transform = carla.Transform()
lane_inv_transform = carla.Transform()

camera_sensor = world_.spawn_actor(camera_bp_, camera_transform, attach_to=vehicle)
collision_sensor = world_.spawn_actor(collision_bp, collision_transform, attach_to=vehicle)
lane_invasion_sensor = world_.spawn_actor(lane_inv_bp, lane_inv_transform, attach_to=vehicle)

In [7]:
camera_sensor.listen(lambda data: image_callback(data))
collision_sensor.listen(lambda data: collision_callback(data))
lane_invasion_sensor.listen(lambda data: lane_invasion_callback(data))

In [15]:
for vehicle in world_.get_actors().filter("*vehicle*"):
    print(vehicle)

Actor(id=24, type=vehicle.mini.cooper_s)


In [8]:
import cv2

In [ ]:
while True:
    cv2.imshow("as", rgb_image)
    cv2.waitKey(1)
    

In [14]:
cv2.destroyAllWindows()